# Notebook: pyolimp - Simulation Examples

In [ ]:
import torch
from torch import Tensor
from matplotlib import pylab as plt
from math import pi

In [ ]:
def show_one(image: Tensor, title: str) -> None:
    if image.isnan().any():
        raise ValueError("has nan")
    fig, ax1 = plt.subplots(dpi=72, figsize=(6, 4.5), ncols=1, nrows=1)
    plt.title(title)
    ax1.imshow(image)

* SCA Kernel

In [ ]:
from olimp.simulate.psf_sca import PSFSCA


params = {
    "sphere_dpt": -1.0,
    "cylinder_dpt": 0.0,
    "angle_rad": 0.0,
    "pupil_diameter_mm": 4.0,
    "am2px": 0.001,
}

params1 = {**params}
show_one(PSFSCA(64, 64)(**params1), f"{params1}")
params2 = {**params, "cylinder_dpt": 3}
show_one(PSFSCA(64, 64)(**params2), f"{params2}")
params3 = {**params, "cylinder_dpt": -2, "angle_rad": pi / 1.5}
show_one(PSFSCA(64, 64)(**params3), f"{params3}")

* Gauss Kernel

In [ ]:
from olimp.simulate.psf_gauss import PSFGauss

params = {
    "center_x": 32,
    "center_y": 32,
    "theta": 0.0,
    "sigma_x": 5.0,
    "sigma_y": 5.0,
}

show_one(PSFGauss(64, 64)(**params), f"{params}")
params2 = {**params, "sigma_x": 6.0, "sigma_y": 3.0}
show_one(PSFGauss(64, 64)(**params2), f"{params2}")
params3 = {**params, "sigma_x": 6.0, "sigma_y": 3.0, "theta": pi / 5}
show_one(PSFGauss(64, 64)(**params3), f"{params3}")

* Refraction Distortion

In [ ]:
from olimp.simulate.refraction_distortion import RefractionDistortion
from olimp.simulate._demo_distortion import demo


def demo_simulate():
    psf1 = PSFSCA(512, 512)()
    yield RefractionDistortion()(torch.fft.fftshift(psf1)), "psf1"

    psf2 = PSFSCA(512, 512)(sphere_dpt=-3, cylinder_dpt=-2)
    yield RefractionDistortion()(torch.fft.fftshift(psf2)), "psf2"


demo("RefractionDistortion", demo_simulate, on="horse.jpg", size=(512, 512))

* Color Blindness Distortion

In [ ]:
from olimp.simulate.color_blindness_distortion import ColorBlindnessDistortion


def demo_simulate():
    yield ColorBlindnessDistortion.from_type("protan")(), "protan"
    yield ColorBlindnessDistortion.from_type("deutan")(), "deutan"
    yield ColorBlindnessDistortion.from_type("tritan")(), "tritan"


demo("ColorBlindnessDistortion", demo_simulate)